In [ ]:
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib oauth2client -t .
# pip install oauth2client 
# pip install html2text

In [1]:
import base64
from apiclient import errors

def GetAttachments(service, user_id, msg_id):
    """Get and store attachment from Message with given id.

    :param service: Authorized Gmail API service instance.
    :param user_id: User's email address. The special value "me" can be used to indicate the authenticated user.
    :param msg_id: ID of Message containing attachment.
    """
    try:
        folder_name = 'Email_Attachments'
        parent_dir = '/home/dell/Documents'
        if 'parts' in messageDetailPayload:
            for msgPayLoad in messageDetailPayload['parts']:
                mime_type = msgPayload['mimeType']
                file_name = msgPayload['filename']
                body = msgPayload['body']
                
                if 'attachmentId' in body:
                    attachment_id = body['attachmentId']
                    
                    response = service.users().messages().attachments().get(
                        userId='me',
                        messageId=email_message['id'],
                        id=attachment_id).execute()
                    file_data = base64.urlsafe_b64decode(
                        response.get('data').encode('UTF-8'))
#                     name_of_file = xxxxx
                    save_path = create_folder(folder_name, parent_dir)
                    
#                     path = os.path.join(save_path, name_of_file+".txt") 
                    with open(save_path, 'w') as f:
                        f.write(file_data)
                    
                    
        
        
#         message = service.users().messages().get(userId=user_id, id=msg_id).execute()

#         for part in message['payload']['parts']:
#             if part['filename']:
#                 if 'data' in part['body']:
#                     data = part['body']['data']
#                 else:
#                     att_id = part['body']['attachmentId']
#                     att = service.users().messages().attachments().get(userId=user_id, messageId=msg_id,id=att_id).execute()
#                     data = att['data']
#                 file_data = base64.urlsafe_b64decode(data.encode('UTF-8'))
#                 path = part['filename']

#                 with open(path, 'w') as f:
#                     f.write(file_data)

    except Exception as e:
        print (e)

In [2]:
def search_email(service, query_string, label_ids=[]):
    try:
        message_list_response = service.users().messages().list(
            userId= 'me',
            labelIds = label_ids,
            q=query_string
        ).execute()
        
        message_items = message_list_response.get('messages')
        nextPageToken = message_list_response.get('nextPageToken')
        
        while nextPageToken:
            message_list_response = service.users().messages().list(
                userId= 'me',
                labelIds = label_ids,
                q=query_string,
                pageToken=nextPageToken
            ).execute()
            message_items.extend(message_list_response.get('messages'))
            nextPageToken = message_list_response.get('nextPageToken')
        return message_items
    
    
    except Exception as e:
        return None

In [3]:
def get_message_detail(service, message_id, format='metadata', metadata_headers=[]):
    try:
        message_detail = service.users().messages().get(
            userId= 'me',
            id= message_id,
            format= format,
            metadataHeaders= metadata_headers
            ).execute()
        return message_detail
        
    except Exception as e:
        print(e)
        return None
    

In [4]:
import os
  
def create_folder(folder_name, parent_dir):
    
#     directory = "Email_Attachemnts"

#     parent_dir = "/home/dell/Documents"
    path = os.path.join(parent_dir, folder_name)
    

    # Create the directory
    # 'GeeksForGeeks' in
    # '/home / User / Documents'
    os.mkdir(path)
    return path

In [5]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        print(creds)
        # Call the Gmail API
        service = build('gmail', 'v1', credentials=creds)
#         results = service.users().labels().list(userId='me').execute()
#         labels = results.get('labels', [])

#         if not labels:
#             print('No labels found.')
#             return
#         print('Labels:')
#         for label in labels:
#             print(label['name'])
        query_string = 'has:attachment from:noreply@swiggy.in'
        email_messages = search_email(service, query_string, ['INBOX'])
#         print(email_messages)
        count = 0
        folder_name = 'Email_Attachments'
        parent_dir = os.path.abspath(os.curdir)
        save_path = create_folder(folder_name, parent_dir)
        for email_message in email_messages:
            count = count+1
            messageId = email_message['threadId']
            messageSubject = '(No Subject) ({0})'.format(messageId)
            messageDetail = get_message_detail(service, email_message['id'], format='full', metadata_headers=['parts'])
            messageDetailPayload = messageDetail.get('payload')
#             messageDetailPayload['headers']
#            print(messageDetailPayload)
       
        
        

            if 'parts' in messageDetailPayload:
                for msgPayload in messageDetailPayload['parts']:
                    mime_type = msgPayload['mimeType']
                    file_name = msgPayload['filename'][5:]
                    print(file_name)
                    body = msgPayload['body']

                    if 'attachmentId' in body:
                        attachment_id = body['attachmentId']

                        response = service.users().messages().attachments().get(
                            userId='me',
                            messageId=email_message['id'],
                            id=attachment_id).execute()
                        file_data = base64.urlsafe_b64decode(
                            response.get('data').encode('UTF-8'))
        #                     name_of_file = xxxxx
                        

                        updated_path = os.path.join(save_path, file_name) 

                        curpath = os.path.abspath(os.curdir)
        #                     print ("Current path is: %s" % (curpath))
        #                     print ("Trying to open: %s" % (os.path.join(curpath, file_name)))


                        with open(updated_path, 'wb') as f:
                            f.write(file_data)
        print(count)  
    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f'An error occurred: {error}')


if __name__ == '__main__':
    main()


0522703021500001_956005f2-024f-4ea2-ac80-8ebcb45018b9.pdf

0263185020100005_a6c65a12-5f77-4394-a258-7d521d723f8c.pdf

0522703013100013_803ac311-6a39-4ad9-9599-a1c9698628a3.pdf

0120193013000009_909b81c4-7040-4310-8470-3823e8a55655.pdf

0546139012600006_f2a527d4-0d4b-4913-8144-36520021a25e.pdf

0152804011600013_72e13021-e506-4ee0-b5fa-e51a0b8d3132.pdf

0131874122400046_3834af57-244b-4c01-9ed2-9c287a2a39fa.pdf

0525151120900008_dfe57aee-c0fc-45bf-b676-5fbc98c942f9.pdf

0470644120400002_17ef0bfe-c359-4ed5-9d6f-b882534197d4.pdf

0126738112600001_76001048-7bf1-414a-a875-ce5e72c6c8c8.pdf

0522703112300003_5a377798-4221-4031-aba0-b9281fdae274.pdf

0522703112200002_f80bc6e3-bd26-4e96-8661-47769d921a6d.pdf

0593264111400005_3caf2ff3-900f-4204-93f7-257170e4becd.pdf

0164297111300001_323b9cab-8073-43fb-9dc1-304a0ece9f7b.pdf

0120193110400003_9f145af4-c57e-46ea-a93d-5381a2cbe129.pdf

0167678092900045_f17df6a0-6aa8-4f94-8e54-42de04e99db3.pdf

0326043081600021_8a1490f2-75b6-476b-acbf-ac3db3d34c03.p